# RNN's, para que sirven?

*Datos de series temporales. Cuando desea predecir el comportamiento futuro basado en el comportamiento pasado. Registros web, registros de sensores, operaciones bursátiles. Dónde conducir su casr autónomo basado en trayectorias pasadas.  Datos que consisten en secuencias de longitud arbitraria. Traducción automática. Leyendas de las imágenes. Música generada por máquina*

**Topologias**

*Sequence to sequence, es decir, predecir los precios de las acciones en función de series de datos históricos. Sequence to vector, es decir, palabras en una oración a sentimiento. Vector to sequence, es decir, crear leyendas a partir de una imagen. Coder -> Decoder, Sequence -> vector-> sequence . es decir, traducción automática*


**Entrenando RNN,s**

*Retropropagación a través del tiempo. al igual que la propagación hacia atrás en MLP, pero aplicada a cada paso de tiempo. Todos esos pasos de tiempo se suman rápidamente. Termina pareciendo una red neuronal real muy, muy profunda. Puede limitar la propagación inversa a un número limitado de pasos de tiempo (retropropagación truncada a través del tiempo)*

*Los pasos de tiempo anteriores se diluyen con el tiempo. Esto puede ser un problema, por ejemplo, al aprender estructuras de oraciones. Célula LSTM. Célula de memoria a corto plazo. Mantiene estados separados a corto y largo plazo. Célula GRU. Unidad recurrente cerrada. Célula LSTM simplificada que funciona igual de bien*

*Es realmente difícil. Muy sensible a topologías, elección de hiperparámetros. Muy intensivo en recursos. Una elección equivocada puede conducir a una RNN que no converge en absoluto*

# Ejercicio

*Análisis de sentimiento de las críticas de películas*

*Este cuaderno está inspirado en el imdb_lstm.py ejemplo que se envía con Keras.*

*En realidad, es un gran ejemplo del uso de RNN. El conjunto de datos que estamos utilizando consiste en reseñas de películas generadas por el usuario y la clasificación de si al usuario le gustó la película o no en función de su calificación asociada.*

*¡Así que vamos a usar un RNN para hacer análisis de sentimiento en reseñas de películas de texto completo!*

*Piensa en lo increíble que es esto. Vamos a entrenar una red neuronal artificial para "leer" reseñas de películas y adivinar si al autor le gustó la película o no.*

*Dado que comprender el lenguaje escrito requiere realizar un seguimiento de todas las palabras en una oración, necesitamos una red neuronal recurrente para mantener una "memoria" de las palabras que han venido antes a medida que "lee" oraciones a lo largo del tiempo.*

*En particular, usaremos celdas LSTM (memoria larga a corto plazo) porque realmente no queremos "olvidar" palabras demasiado rápido: las palabras al principio de una oración pueden afectar significativamente el significado de esa oración.*

*Comencemos importando las cosas que necesitamos:*

In [1]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

*Ahora importamos nuestros datos de entrenamiento y pruebas. Especificamos que solo nos preocupamos por las 20,000 palabras más populares en el conjunto de datos para mantener las cosas algo manejables. El conjunto de datos incluye 5,000 revisiones de capacitación y 25,000 revisiones de pruebas por alguna razón.*

In [2]:
print('Cargando data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=20000)

Cargando data...
17464789/17464789 [==============================] - 11s 1us/step


*Vamos a tener una idea de cómo se ven estos datos. Veamos la primera función de entrenamiento, que debería representar una reseña escrita de la película:*

In [3]:
x_train[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 19193,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 10311,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 12118,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,

*¡Eso no parece una crítica de película! Pero este conjunto de datos le ha ahorrado muchos problemas: ya han convertido palabras en índices basados en enteros. Las letras reales que componen una palabra realmente no importan en lo que respecta a nuestro modelo, lo que importa son las palabras mismas, y nuestro modelo necesita números para trabajar, no letras.*

*Así que ten en cuenta que cada número en las características de entrenamiento representa alguna palabra específica. Sin embargo, es un fastidio que no podamos simplemente leer las reseñas en inglés como una verificación visceral para ver si el análisis de sentimientos realmente está funcionando.*

*¿Qué aspecto tienen las etiquetas?*

In [4]:
y_train[0]

1

*Son solo 0 o 1, lo que indica si el crítico dijo que le gustó la película o no.*

*Entonces, para recapitular, tenemos un montón de reseñas de películas que se han convertido en vectores de palabras representadas por enteros, y una clasificación de sentimiento binario de la que aprender.*

*Las RNN pueden explotar rápidamente, así que de nuevo para mantener las cosas manejables en nuestra pequeña PC, limitemos las revisiones a sus primeras 80 palabras:*

In [5]:
x_train = sequence.pad_sequences(x_train, maxlen=80)
x_test = sequence.pad_sequences(x_test, maxlen=80)

*¡Ahora configuremos nuestro modelo de red neuronal! Teniendo en cuenta lo complicada que es una red neuronal recurrente LSTM bajo el capó, es realmente sorprendente lo fácil que es hacerlo con Keras.*

*Comenzaremos con una capa de incrustación: este es solo un paso que convierte los datos de entrada en vectores densos de tamaño fijo que se adaptan mejor a una red neuronal. Por lo general, esto se ve junto con datos de texto basados en índices como los que tenemos aquí. El 20,000 indica el tamaño del vocabulario (recuerde que dijimos que solo queríamos las 20,000 palabras principales) y 128 es la dimensión de salida de 128 unidades.*

*A continuación, solo tenemos que configurar una capa LSTM para el propio RNN. Así de fácil. Especificamos 128 para que coincida con el tamaño de salida de la capa de incrustación, y los términos de abandono para evitar el sobreajuste, a lo que los RNN son particularmente propensos.*

*Finalmente, solo necesitamos reducirlo a una sola neurona con una función de activación sigmoide para elegir nuestra clasificación de sentimiento binay de 0 o 1.*

In [6]:
model = Sequential()
model.add(Embedding(20000, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

*Como este es un problema de clasificación binaria, usaremos la función de pérdida de binary_crossentropy. Y el optimizador Adam suele ser una buena opción (siéntase libre de probar otros).*

In [7]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

*Ahora realmente entrenaremos nuestro modelo. Las RNN, como las de CNN, son muy pesadas en recursos. Mantener el tamaño del lote relativamente pequeño es la clave para permitir que esto se ejecute en su PC. En la palabra real, por supuesto, estaría aprovechando las GPU instaladas en muchas computadoras en un clúster para mejorar mucho esta escala*.

## Advertencia

*Esto tardará mucho tiempo en ejecutarse, ¡incluso en una PC rápida! No ejecute los siguientes bloques a menos que esté preparado para atar su computadora durante una hora o más.*

*Ahora comencemos el entrenamiento. ¡Incluso con una GPU, esto llevará mucho tiempo!*

In [8]:
model.fit(x_train, y_train,
          batch_size=32,
          epochs=15,
          verbose=2,
          validation_data=(x_test, y_test))

Epoch 1/15
782/782 - 139s - loss: 0.4367 - accuracy: 0.7934 - val_loss: 0.3579 - val_accuracy: 0.8380 - 139s/epoch - 178ms/step
Epoch 2/15
782/782 - 131s - loss: 0.2560 - accuracy: 0.9004 - val_loss: 0.3808 - val_accuracy: 0.8384 - 131s/epoch - 168ms/step
Epoch 3/15
782/782 - 131s - loss: 0.1657 - accuracy: 0.9366 - val_loss: 0.4690 - val_accuracy: 0.8276 - 131s/epoch - 167ms/step
Epoch 4/15
782/782 - 130s - loss: 0.1103 - accuracy: 0.9598 - val_loss: 0.5585 - val_accuracy: 0.8247 - 130s/epoch - 166ms/step
Epoch 5/15
782/782 - 129s - loss: 0.0686 - accuracy: 0.9778 - val_loss: 0.7360 - val_accuracy: 0.8194 - 129s/epoch - 165ms/step
Epoch 6/15
782/782 - 128s - loss: 0.0515 - accuracy: 0.9824 - val_loss: 0.6593 - val_accuracy: 0.8202 - 128s/epoch - 164ms/step
Epoch 7/15
782/782 - 128s - loss: 0.0374 - accuracy: 0.9872 - val_loss: 0.8509 - val_accuracy: 0.8124 - 128s/epoch - 164ms/step
Epoch 8/15
782/782 - 157s - loss: 0.0275 - accuracy: 0.9919 - val_loss: 0.7934 - val_accuracy: 0.8172 - 

*Bien, evaluemos la precisión de nuestro modelo:*

In [9]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=32,
                            verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 - 16s - loss: 1.2783 - accuracy: 0.8010 - 16s/epoch - 21ms/step
Test score: 1.2783071994781494
Test accuracy: 0.8009999990463257


*80% ¿eh? No está mal, teniendo en cuenta que nos limitamos a solo las primeras 80 palabras de cada revisión.*

*Tenga en cuenta que la precisión de validación mientras estábamos entrenando nunca mejoró realmente después de la primera época; Es probable que estemos sobreadaptados. Este es un caso en el que la interrupción temprana habría sido beneficiosa.*

*Pero de nuevo, ¡detente y piensa en lo que acabamos de hacer aquí! Una red neuronal que puede "leer" reseñas y deducir si al autor le gustó la película o no basándose en ese texto. Y tiene en cuenta el contexto de cada palabra y su posición en la revisión, ¡y configurar el modelo en sí fue solo unas pocas líneas de código! Es bastante increíble lo que puedes hacer con Keras.*